### Import Libraries

In [64]:
import csv
import string 
import re
import ast
import itertools
import pandas as ps 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter
from itertools import combinations, permutations
from cleantext import clean   # !pip install clean-text
ps.options.mode.chained_assignment = None

### Read Data file containing data about Datasets

In [173]:
column_names = ['title','subtitle','description','keywords','url']

In [174]:
df = ps.read_csv('D:/ODASE/Data/dataset.csv')
#df.drop(columns=['image','pubdate','video','rank'],axis=1,inplace = True)

In [175]:
df.columns = column_names
df = df.fillna("")
df.drop(index = 1037 , inplace=True)


In [176]:
df.head()

,title,subtitle,description,keywords,url
0,b'Wine Reviews',"b'130k wine reviews with variety, location, wi...","b""### Context\n\nAfter watching [Somm](http://...","b""['food and drink', 'critical theory', 'mediu...",https://www.kaggle.com/zynicide/wine-reviews
1,b'UFO Sightings + Air Quality',b'UFO (USA / lights) + Air Quality (USA / Poll...,b'This dataset is the result of the fusion of ...,"b""['climate', 'space', 'earth sciences', 'time...",https://www.kaggle.com/infof422henni/ufo-air-q...
2,b'Safebooru - Anime Image Metadata',b'1.9 million rows of tag-based anime image me...,b'### Context\n\n[Safebooru][1] is a tag-based...,"b""['popular culture', 'visual arts', 'animatio...",https://www.kaggle.com/alamson/safebooru
3,b'QuickDraw Sketches',b'Sketches and Strokes from the QuickDraw Game',b'### Context\n\nThe dataset consists of the s...,"b""['image processing', 'visual arts', 'drawing...",https://www.kaggle.com/google/tinyquickdraw
4,b'EEG Micro-experiment',b'Music vs. Reading (3-4 min per condition)',"b""### Context\n\nThis is a tiny self-experimen...","b""['neuroscience', 'biotechnology', 'small', '...",https://www.kaggle.com/millerintllc/eeg-microe...


In [177]:
len(df)

4620

In [180]:
def cleanb(text):
    if text is not "":
        return ast.literal_eval(text)
    return []

<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\prasa\AppData\Local\Temp/ipykernel_1684/4237603897.py:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if text is not "":


In [182]:
df['title'] = df['title'].apply(lambda x:re.sub(r'\bnan\b', "", str(x), 0, re.MULTILINE))
df['title'] = df['title'].apply(lambda x:re.sub(r'^b\'(.*)\'', "\\g<1>", str(x), 0, re.MULTILINE))

df['subtitle'] = df['subtitle'].apply(lambda x:re.sub(r'\bnan\b', "", str(x), 0, re.MULTILINE))
df['subtitle'] = df['subtitle'].apply(lambda x:re.sub(r'^b\'(.*)\'', "\\g<1>", str(x), 0, re.MULTILINE))

df['description'] = df['description'].apply(lambda x: re.sub(r'\bnan\b', "", str(x), 0, re.MULTILINE))
df['description'] = df['description'].apply(lambda x: re.sub(r'^b[\'|\"](.*)[\' | \"]', "\\g<1>", str(x), 0, re.MULTILINE))
df['description'] = df['description'].apply(lambda x: x.replace("\\n","").replace("### Context\\n\\n","").replace("#",""))
df['description'] = df['description'].apply(lambda x: x.replace("Context","").replace("context","").replace("Content","").replace("content",""))

df['keywords'] = df['keywords'].apply(lambda x: re.sub(r'\bnan\b', "[]", str(x), 0, re.MULTILINE))
df['keywords'] = df['keywords'].apply(lambda x:re.sub(r'^b\"(.*)\"', "\\g<1>", str(x), 0, re.MULTILINE))
df['keywords'] = df['keywords'].apply(lambda x: cleanb(x))
# df['keywords'] = df['keywords'].apply(lambda x: ast.literal_eval(x))

df.head()

,title,subtitle,description,keywords,url
0,Wine Reviews,"130k wine reviews with variety, location, wine...",After watching [Somm](http://www.imdb.com/tit...,"[food and drink, critical theory, medium, feat...",https://www.kaggle.com/zynicide/wine-reviews
1,UFO Sightings + Air Quality,UFO (USA / lights) + Air Quality (USA / Pollut...,This dataset is the result of the fusion of tw...,"[climate, space, earth sciences, timelines, na...",https://www.kaggle.com/infof422henni/ufo-air-q...
2,Safebooru - Anime Image Metadata,1.9 million rows of tag-based anime image meta...,[Safebooru][1] is a tag-based image archive m...,"[popular culture, visual arts, animation, draw...",https://www.kaggle.com/alamson/safebooru
3,QuickDraw Sketches,Sketches and Strokes from the QuickDraw Game,The dataset consists of the series of strokes...,"[image processing, visual arts, drawing, large...",https://www.kaggle.com/google/tinyquickdraw
4,EEG Micro-experiment,Music vs. Reading (3-4 min per condition),This is a tiny self-experiment. The researche...,"[neuroscience, biotechnology, small, featured]",https://www.kaggle.com/millerintllc/eeg-microe...


In [183]:
df = df.reset_index()

### Encoding URLS of Documents to Numeric scale 

In [184]:
#Encoding URL's into unique ID's
label_encoder = LabelEncoder()
df['url_index'] = label_encoder.fit_transform(df['url'])

In [185]:
len(df)

4620

### Preprocessing of Source, Title and Description

In [186]:
# Conversion to Lower case.

df['title'] = df['title'].str.lower()
df['subtitle'] = df['subtitle'].str.lower()
df['description'] = df['description'].str.lower()

#Stripped the string.

df['title'] = df['title'].str.strip()
df['subtitle'] = df['subtitle'].str.strip()
df['description'] = df['description'].str.strip()

### Clean text in title and description

In [187]:
df['title'] = df.apply(lambda row: clean(row['title'],fix_unicode=True,no_urls=True,to_ascii=True,lower=True,no_emails=True,),axis=1)
df['subtitle'] = df.apply(lambda row: clean(row['subtitle'],fix_unicode=True,no_urls=True,to_ascii=True,lower=True,no_emails=True,),axis=1)
df['description'] = df.apply(lambda row: clean(row['description'],fix_unicode=True,no_urls=True,to_ascii=True,lower=True,no_emails=True),axis=1)

### Remove Punctuations

In [188]:
df['title'] = df['title'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
df['subtitle'] = df['subtitle'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
df['description'] = df['description'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))

### Tokenization

In [189]:
df['tokenized_title'] = df.apply(lambda row: word_tokenize(row['title']),axis=1) 
df['tokenized_subtitle'] = df.apply(lambda row: word_tokenize(row['subtitle']),axis=1)  
df['tokenized_description'] = df.apply(lambda row: word_tokenize(row['description']),axis=1)

### Combine tokens from title and description of a document 

In [218]:
df['doc'] = df['tokenized_title'] + df['tokenized_subtitle'] + df['tokenized_description']

### List of tokens in a doc

In [219]:
df['doc']

0       [wine, reviews, 130k, wine, reviews, with, var...
1       [ufo, sightings, air, quality, ufo, usa, light...
2       [safebooru, anime, image, metadata, 19, millio...
3       [quickdraw, sketches, sketches, and, strokes, ...
4       [eeg, microexperiment, music, vs, reading, 34,...
                              ...                        
4615                          [selu, with, good, init, 2]
4616                                [price, per, product]
4617    [japan, trade, stats, custom, 2016, data, sub,...
4618                            [us, flights, data, 2008]
4619                                [iowa, house, prices]
Name: doc, Length: 4620, dtype: object

### Stopword & Non-alphanumeric characters removal

In [220]:
#stop word, digits,non-english alphabets removal
stop_words = set(stopwords.words('english'))
for i in range(len(df)):
    df['doc'][i] = [w for w in df['doc'][i] if not w in stop_words]
    df['doc'][i] = list(filter(lambda w: re.search("^[a-zA-Z]{3,}$", w) is not None, df['doc'][i]))
    df['doc'][i] = [w for w in df['doc'][i] if not w.isdigit()] # Remove digits

### Removal of Pronouns

In [221]:
# Removal of Pronouns (Singular and Plural)
for i in range(len(df)):
    tagged_doc = pos_tag(df.doc[i])
    edited_doc = [word for word,tag in tagged_doc if tag!= 'NNP' and tag!= 'NNPS']
    df.doc[i] = edited_doc

### Stemming & Lemmatization of tokens 

In [222]:
#Stemming and lemmatization
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
for i in range(len(df)):
    for j in range(len(df['doc'][i])):
        df['doc'][i][j] = lemmatizer.lemmatize(df['doc'][i][j])
        df['doc'][i][j] = stemmer.stem(df['doc'][i][j])

In [223]:
for i in range(len(df)):
    df['doc'][i] = [w for w in df['doc'][i] if not w in stop_words]
    df['doc'][i] = list(filter(lambda w: re.search("^[a-zA-Z]{3,}$", w) is not None, df['doc'][i]))

### Word count in each document

In [224]:
df['word_count'] = df.apply(lambda row: Counter(row['doc']),axis=1)  # Adding in a new column

In [225]:
df['word_count']

0       {'wine': 10, 'review': 7, 'varieti': 3, 'locat...
1       {'ufo': 4, 'sight': 4, 'air': 3, 'qualiti': 3,...
2       {'safebooru': 2, 'anim': 3, 'imag': 7, 'metada...
3       {'quickdraw': 3, 'sketch': 2, 'stroke': 5, 'ga...
4       {'eeg': 3, 'microexperi': 1, 'music': 3, 'read...
                              ...                        
4615                    {'selu': 1, 'good': 1, 'init': 1}
4616                 {'price': 1, 'per': 1, 'product': 1}
4617    {'japan': 2, 'trade': 2, 'stat': 1, 'custom': ...
4618                             {'flight': 1, 'data': 1}
4619                   {'iowa': 1, 'hous': 1, 'price': 1}
Name: word_count, Length: 4620, dtype: object

### Total tokens from the corpus

In [226]:
add_dict = Counter()

for i in range(len(df)):
    counter1 = Counter(df['word_count'][i])
    add_dict += counter1
    
final_dict = dict(add_dict) # Sum of all word_count dictionaries is final dictionary.
#print(final_dict)

### Length of BOW (Bag Of Words)

In [227]:
len(final_dict)

30630

In [228]:
final_dict

{'wine': 37,
 'review': 434,
 'varieti': 113,
 'locat': 735,
 'wineri': 2,
 'price': 544,
 'descript': 674,
 'watch': 49,
 'sommurl': 1,
 'documentari': 4,
 'master': 38,
 'sommeli': 3,
 'wonder': 20,
 'could': 294,
 'creat': 599,
 'predict': 771,
 'model': 812,
 'identifi': 515,
 'blind': 3,
 'tast': 19,
 'like': 765,
 'would': 505,
 'first': 536,
 'step': 70,
 'journey': 10,
 'gather': 135,
 'data': 11465,
 'train': 628,
 'plan': 146,
 'use': 4710,
 'deep': 150,
 'learn': 787,
 'word': 599,
 'descriptionreview': 1,
 'still': 113,
 'wont': 16,
 'abl': 150,
 'theoret': 4,
 'base': 630,
 'give': 141,
 'anyon': 82,
 'idea': 140,
 'accomplish': 19,
 'pleas': 411,
 'post': 409,
 'dataset': 7791,
 'contain': 2174,
 'three': 272,
 'file': 2336,
 'column': 636,
 'row': 309,
 'node': 37,
 'click': 41,
 'tab': 32,
 'see': 654,
 'individu': 630,
 'columnlevel': 3,
 'metadata': 172,
 'summari': 156,
 'statist': 550,
 'acknowledgementsth': 221,
 'scrape': 278,
 'wineenthusiasturl': 1,
 'week': 161

### Bag of Words Construction

In [229]:
bag_of_words = []
for k, v in list(final_dict.items()):
    bag_of_words.append(k)

In [230]:
len(bag_of_words)

30630

In [231]:
bag_of_words = np.array(bag_of_words)
bag_of_words = np.unique(bag_of_words)

In [232]:
final_bag = dict()
for i in range(len(bag_of_words)):
    final_bag[bag_of_words[i]] = i

### Saving bag of words as array object

In [233]:
with open('./Data/bag_of_words.npy', 'wb') as f:
    np.save(f,bag_of_words,allow_pickle=True)

In [234]:
df.to_csv('./Data/data_clean.csv')

In [244]:
df_clean = ps.read_csv('./Data/data_clean.csv',index_col=0)

In [245]:
df.columns

Index(['index', 'title', 'subtitle', 'description', 'keywords', 'url',
       'url_index', 'tokenized_title', 'tokenized_subtitle',
       'tokenized_description', 'doc', 'word_count'],
      dtype='object')

In [247]:
df.drop(["index"],axis=1)

,title,subtitle,description,keywords,url,url_index,tokenized_title,tokenized_subtitle,tokenized_description,doc,word_count
0,wine reviews,130k wine reviews with variety location winery...,after watching sommurl a documentary on master...,"[food and drink, critical theory, medium, feat...",https://www.kaggle.com/zynicide/wine-reviews,4179,"[wine, reviews]","[130k, wine, reviews, with, variety, location,...","[after, watching, sommurl, a, documentary, on,...","[wine, review, wine, review, varieti, locat, w...","{'wine': 10, 'review': 7, 'varieti': 3, 'locat..."
1,ufo sightings air quality,ufo usa lights air quality usa pollutants l...,this dataset is the result of the fusion of tw...,"[climate, space, earth sciences, timelines, na...",https://www.kaggle.com/infof422henni/ufo-air-q...,1648,"[ufo, sightings, air, quality]","[ufo, usa, lights, air, quality, usa, pollutan...","[this, dataset, is, the, result, of, the, fusi...","[ufo, sight, air, qualiti, ufo, usa, light, ai...","{'ufo': 4, 'sight': 4, 'air': 3, 'qualiti': 3,..."
2,safebooru anime image metadata,19 million rows of tagbased anime image metadata,safebooru1 is a tagbased image archive maintai...,"[popular culture, visual arts, animation, draw...",https://www.kaggle.com/alamson/safebooru,169,"[safebooru, anime, image, metadata]","[19, million, rows, of, tagbased, anime, image...","[safebooru1, is, a, tagbased, image, archive, ...","[safebooru, anim, imag, metadata, million, row...","{'safebooru': 2, 'anim': 3, 'imag': 7, 'metada..."
3,quickdraw sketches,sketches and strokes from the quickdraw game,the dataset consists of the series of strokes ...,"[image processing, visual arts, drawing, large...",https://www.kaggle.com/google/tinyquickdraw,1465,"[quickdraw, sketches]","[sketches, and, strokes, from, the, quickdraw,...","[the, dataset, consists, of, the, series, of, ...","[quickdraw, sketch, sketch, stroke, quickdraw,...","{'quickdraw': 3, 'sketch': 2, 'stroke': 5, 'ga..."
4,eeg microexperiment,music vs reading 34 min per condition,this is a tiny selfexperiment the researcher c...,"[neuroscience, biotechnology, small, featured]",https://www.kaggle.com/millerintllc/eeg-microe...,2227,"[eeg, microexperiment]","[music, vs, reading, 34, min, per, condition]","[this, is, a, tiny, selfexperiment, the, resea...","[eeg, microexperi, music, read, min, per, cond...","{'eeg': 3, 'microexperi': 1, 'music': 3, 'read..."
...,...,...,...,...,...,...,...,...,...,...,...
4615,selu with good init 2,,,[small],https://www.kaggle.com/hypdeb/selu-with-good-i...,1615,"[selu, with, good, init, 2]",[],[],"[selu, good, init]","{'selu': 1, 'good': 1, 'init': 1}"
4616,price per product,,,[small],https://www.kaggle.com/azizabidi/price-per-pro...,391,"[price, per, product]",[],[],"[price, per, product]","{'price': 1, 'per': 1, 'product': 1}"
4617,japan trade stats custom 2016 data,sub set of japantradestatistics,sub dataset of japan trade statistics url 2016...,"[finance, business, medium]",https://www.kaggle.com/zanjibar/custom-2016,4136,"[japan, trade, stats, custom, 2016, data]","[sub, set, of, japantradestatistics]","[sub, dataset, of, japan, trade, statistics, u...","[japan, trade, stat, custom, data, sub, set, j...","{'japan': 2, 'trade': 2, 'stat': 1, 'custom': ..."
4618,us flights data 2008,,,[medium],https://www.kaggle.com/vikalpdongre/us-flights...,3968,"[us, flights, data, 2008]",[],[],"[flight, data]","{'flight': 1, 'data': 1}"


In [249]:
df[['url_index','url','doc']]

,url_index,url,doc
0,4179,https://www.kaggle.com/zynicide/wine-reviews,"[wine, review, wine, review, varieti, locat, w..."
1,1648,https://www.kaggle.com/infof422henni/ufo-air-q...,"[ufo, sight, air, qualiti, ufo, usa, light, ai..."
2,169,https://www.kaggle.com/alamson/safebooru,"[safebooru, anim, imag, metadata, million, row..."
3,1465,https://www.kaggle.com/google/tinyquickdraw,"[quickdraw, sketch, sketch, stroke, quickdraw,..."
4,2227,https://www.kaggle.com/millerintllc/eeg-microe...,"[eeg, microexperi, music, read, min, per, cond..."
...,...,...,...
4615,1615,https://www.kaggle.com/hypdeb/selu-with-good-i...,"[selu, good, init]"
4616,391,https://www.kaggle.com/azizabidi/price-per-pro...,"[price, per, product]"
4617,4136,https://www.kaggle.com/zanjibar/custom-2016,"[japan, trade, stat, custom, data, sub, set, j..."
4618,3968,https://www.kaggle.com/vikalpdongre/us-flights...,"[flight, data]"
